Install pandasql.

In [1]:
pip install pandasql

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandasql-0.7.3-py3-none-any.whl
     |████████████████████████████████| 2.0 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 18.4 MB/s eta 0:00:01
     |████████████████████████████████| 10.9 MB 69.8 MB/s eta 0:00:01
     |████████████████████████████████| 341 kB 39.9 MB/s eta 0:00:01
     |████████████████████████████████| 502 kB 35.9 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Import pandas and sqldf.

In [2]:
import pandas as pd
from pandasql import sqldf

Store the data in dataframes

In [4]:
campaign_info = pd.read_csv("campaign_info.csv")
marketing_performance = pd.read_csv("marketing_performance.csv")
website_revenue = pd.read_csv("website_revenue.csv")

1. Write a query to get the sum of impressions by day.

In [43]:
impressions_by_day = sqldf("""
      SELECT date, SUM(impressions) AS total_impressions
      FROM marketing_performance
      GROUP BY date
      ORDER BY date
      """)

impressions_by_day

,date,total_impressions
0,2023-07-24 0:00:00,1423
1,2023-07-26 0:00:00,1547
2,2023-07-27 0:00:00,2295
3,2023-07-28 0:00:00,8142
4,2023-07-29 0:00:00,3511
5,2023-07-30 0:00:00,3001
6,2023-08-02 0:00:00,1532
7,2023-08-03 0:00:00,3189
8,2023-08-04 0:00:00,3254
9,2023-08-05 0:00:00,1088


2. Write a query to get the top three revenue-generating states in order of best to worst. How much revenue did the third best state generate?

In [15]:
top_three_states = sqldf("""
      SELECT state, SUM(revenue) as total_revenue
      FROM website_revenue
      GROUP BY state
      ORDER BY total_revenue DESC
      LIMIT 3
      """)

top_three_states


,state,total_revenue
0,NY,46398
1,GA,39666
2,OH,37577


In [16]:
top_three_states.iloc[2]["total_revenue"]

37577

3. Write a query that shows total cost, impressions, clicks, and revenue of each campaign. Make sure to include the campaign name in the output.

In [44]:
campaign_data = sqldf("""
    SELECT
        B.name AS campaign_name,
        A.campaign_id,   
        SUM(A.cost) AS total_cost,
        SUM(A.impressions) AS total_impressions,
        SUM(A.clicks) AS total_clicks,
        SUM(C.revenue) AS total_revenue
    FROM
        marketing_performance AS A
    LEFT JOIN
        campaign_info AS B ON A.campaign_id = B.id
    LEFT JOIN
        website_revenue AS C ON A.campaign_id = C.campaign_id
    GROUP BY
        B.name, A.campaign_id
    """)

campaign_data

,campaign_name,campaign_id,total_cost,total_impressions,total_clicks,total_revenue
0,Campaign1,64441550,4170.51,42810,38724,151792
1,Campaign2,55304737,4075.50,40938,29652,155308
2,Campaign3,89363211,15809.04,158280,116048,551672
3,Campaign4,86363015,3970.14,47508,33318,163396
4,Campaign5,99058240,4077.15,25641,33663,136404


4. Write a query to get the number of conversions of Campaign5 by state. Which state generated the most conversions for this campaign?

In [41]:
conversion_by_state = sqldf("""
    SELECT
        SUBSTR(A.geo, INSTR(A.geo, '-') + 1) AS state,
        SUM(A.conversions) AS total_conversions
    FROM
        marketing_performance AS A
    LEFT JOIN
        campaign_info AS B ON A.campaign_id = B.id
    WHERE
        B.name = "Campaign5"
    GROUP BY
        state
    ORDER BY
        total_conversions DESC
    """)

conversion_by_state

,state,total_conversions
0,GA,672
1,OH,442


In [42]:
conversion_by_state.iloc[0]["state"]

'GA'

5. In your opinion, which campaign was the most efficient, and why?

In [55]:
campaign_performance = sqldf("""
    SELECT A.*, SUM(B.conversions) AS total_conversions
    FROM campaign_data AS A
    JOIN marketing_performance AS B ON A.campaign_id = B.campaign_id
    GROUP BY A.campaign_id
    ORDER BY A.campaign_name
    """)

campaign_performance

,campaign_name,campaign_id,total_cost,total_impressions,total_clicks,total_revenue,total_conversions
0,Campaign1,64441550,4170.51,42810,38724,151792,3018
1,Campaign2,55304737,4075.50,40938,29652,155308,1516
2,Campaign3,89363211,15809.04,158280,116048,551672,4451
3,Campaign4,86363015,3970.14,47508,33318,163396,1551
4,Campaign5,99058240,4077.15,25641,33663,136404,1114


In [64]:
conversion_rate = sqldf("""
    SELECT
        campaign_name,
        ROUND((total_conversions * 100.0 / total_clicks), 2) AS conversion_rate
    FROM
        campaign_performance
    ORDER BY
        conversion_rate DESC
    """)

conversion_rate

,campaign_name,conversion_rate
0,Campaign1,7.79
1,Campaign2,5.11
2,Campaign4,4.66
3,Campaign3,3.84
4,Campaign5,3.31


In [65]:
cost_per_conversion = sqldf("""
    SELECT
        campaign_name,
        ROUND((total_cost / total_conversions), 2) AS cost_per_conversion
    FROM
        campaign_performance
    ORDER BY
        cost_per_conversion ASC
    """)

cost_per_conversion

,campaign_name,cost_per_conversion
0,Campaign1,1.38
1,Campaign4,2.56
2,Campaign2,2.69
3,Campaign3,3.55
4,Campaign5,3.66


In [67]:
click_through_rate = sqldf("""
    SELECT
        campaign_name,
        ROUND((total_clicks * 100 / total_impressions), 2) AS click_through_rate
    FROM
        campaign_performance
    ORDER BY
        click_through_rate DESC
    """)

click_through_rate

,campaign_name,click_through_rate
0,Campaign5,131.0
1,Campaign1,90.0
2,Campaign3,73.0
3,Campaign2,72.0
4,Campaign4,70.0


In [68]:
return_on_investment = sqldf("""
    SELECT
        campaign_name,
        ROUND(((total_revenue - total_cost) / total_cost) * 100.0, 2) AS return_on_investment
    FROM
        campaign_performance
    ORDER BY
        return_on_investment DESC
    """)

return_on_investment

,campaign_name,return_on_investment
0,Campaign4,4015.62
1,Campaign2,3710.77
2,Campaign1,3539.65
3,Campaign3,3389.60
4,Campaign5,3245.57


In [76]:
print("""From the metrics shown above, Campaign 1 is considered most effective in terms of conversion and cost efficiency.
      \nHowever, in terms of profitability, Camapiagn 4  has the highest ROI.
      \nSince Campaign 5 has a CTR > 100%, which is highly unusual and may indicate issue in the data, we will not take that into our consideration.
      \nIn my opinion, the final return is what matters the most. Therefore, I think Campaign 4 is most effecetive""")

From the metrics shown above, Campaign 1 is considered most effective in terms of conversion and cost efficiency.
      
However, in terms of profitability, Camapiagn 4  has the highest ROI.
      
Since Campaign 5 has a CTR > 100%, which is highly unusual and may indicate issue in the data, we will not take that into our consideration.
      
In my opinion, the final return is what matters the most. Therefore, I think Campaign 4 is most effecetive


6. Write a query that showcases the best day of the week (e.g., Sunday, Monday, Tuesday, etc.) to run ads.

In [120]:
from datetime import datetime

def date_to_day_of_week(date_str):
    date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    return date.strftime('%A')

website_revenue['date'].apply(date_to_day_of_week)

website_revenue

,date,campaign_id,state,revenue,day_of_week
0,2023-07-24 0:00:00,64441550,GA,6370,Monday
1,2023-08-02 0:00:00,89363211,CA,6676,Wednesday
2,2023-08-03 0:00:00,86363015,CA,6239,Thursday
3,2023-07-24 0:00:00,55304737,TX,6607,Monday
4,2023-08-17 0:00:00,86363015,NY,7014,Thursday
5,2023-08-13 0:00:00,99058240,GA,6765,Sunday
6,2023-07-30 0:00:00,55304737,TX,6803,Sunday
7,2023-08-07 0:00:00,89363211,NY,5600,Monday
8,2023-07-27 0:00:00,86363015,CA,6920,Thursday
9,2023-08-09 0:00:00,55304737,NY,6406,Wednesday


In [122]:
average_revenue = sqldf("""
    SELECT
        day_of_week,
        CAST(AVG(revenue) AS INTEGER) AS average_revenue
    FROM
        website_revenue
    GROUP BY
        day_of_week
    ORDER BY
        average_revenue DESC
    """)

average_revenue

,day_of_week,average_revenue
0,Sunday,6784
1,Wednesday,6695
2,Tuesday,6614
3,Thursday,6594
4,Saturday,6450
5,Friday,6394
6,Monday,6130


In [123]:
average_revenue.iloc[0]

day_of_week        Sunday
average_revenue      6784
Name: 0, dtype: object